In [1]:
# Import required packages

from xcube_sh.config import CubeConfig
from xcube_sh.cube import open_cube
from shapely import geometry
from xcube_sh.sentinelhub import SentinelHub
import xarray as xr
import json
import IPython.display
import shapely.geometry

from sentinelhub import BBox, WmsRequest, DataSource, SHConfig
from functools import partial

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [2]:
# Set Sentinel Hub credentials

import os
sh_credentials = dict(client_id=os.environ['SH_CLIENT_ID'],
                      client_secret=os.environ['SH_CLIENT_SECRET']) # This is only provided when the Oauth credentials are created

# Sentinel-3 OLCI, Sentinel-3 SLSTR and Sentinel-5 layers are processed on different infrastructure, 
# which requires to used different end-point

sh_credentials.update(api_url='https://creodias.sentinel-hub.com') 

In [3]:
f=open('countries.json')
data=json.load(f)
final=dict()
for i in range(0,len(data['features'])):
    #print(data['features'][i]['id'])
    final[data['features'][i]['id']]=data['features'][i]['geometry']['coordinates']

In [4]:
EU_countries={'AUT','BEL','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC',
'HUN','ITA','LVA','LTU','LUX','MLT','NLD','POL','PRT','ROU','SVK','SVN','ESP',
'SWE','ALB','ARM','BLR','BIH','GEO','ISL',
'MKD','MDA','MCO','MNE','NOR','RUS','SRB','CHE','TUR','UKR','GBR'}

In [5]:
aoi = pd.DataFrame()
for country in EU_countries:
    if len(final[country])!=1:
        xs=0
        ys=0
        all_x=[]
        all_y=[]
        for i in range(0,len(final[country])):
            state=geometry.Polygon(final[country][i][0])
            geom = np.array(state.exterior.coords.xy)
            xs = geom[0]
            ys = geom[1]
            all_x.extend(xs)
            all_y.extend(ys)
    else:
        state=geometry.Polygon(final[country][0])
        geom = np.array(state.exterior.coords.xy)
        all_x = geom[0]
        all_y = geom[1]
    aoi=aoi.append([ [country,
                    min(all_x),
                    min(all_y),
                    max(all_x),
                    max(all_y)]],ignore_index='True')
bbox=(min(all_x),
min(all_y),
max(all_x),
max(all_y))

In [6]:
bbox=(min(all_x),
min(all_y),
max(all_x),
max(all_y))

In [7]:
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

<IPython.display.GeoJSON object>

In [8]:
print(aoi.shape)
print(aoi.head(8))

shape = aoi.shape
nullCount = sum(aoi.isna().sum())
print(f"Shape of 2020 DF: {shape}, Count of Null values: {nullCount}".format(shape , nullCount ))

(43, 5)
     0          1          2          3          4
0  FRA  -4.592350  41.380007   9.560016  51.148506
1  LVA  21.055800  55.615107  28.176709  57.970157
2  LTU  21.055800  53.905702  26.588279  56.372528
3  CHE   6.022609  45.776948  10.442701  47.830828
4  BLR  23.199494  51.319503  32.693643  56.169130
5  MLT  14.180354  35.820191  14.566171  36.075809
6  ISL -24.326184  63.496383 -13.609732  66.526792
7  ARM  43.582746  38.741201  46.505720  41.248129
Shape of 2020 DF: (43, 5), Count of Null values: 0


In [9]:
def caculateNO2(geometry, timerange):
    """
    parameters:
    geometry: BBox object to be passed. This contains the bounding box for the area of interest (AOI)
    timerange: list giving the start & end of the time range format: YYYY-mm-dd
    
    return:
    a dataframe with two columns: Mean NO2 and Timestamp
    NO2 mean values for the time span between given timerange 
    
    """
    
    cube_config = CubeConfig(dataset_name='S5PL2',
                         band_names=['NO2'],
                         tile_size=[512, 512],
                         geometry=geometry,
                         spatial_res=(bbox[2]-bbox[0])/512, # spatial resolution (approx. 20 m in degree)
                         time_range= timerange,
                         time_period='7D') 
    cube = open_cube(cube_config, **sh_credentials)

    no2_values = list() 
    timestamp = list()

    for i in range(cube.time.shape[0]):
        no2_values.append(np.nanmean(cube.NO2.isel(time=i).values[0]))
        timestamp.append(cube.NO2.isel(time=i).time.values)
        
    assert len(no2_values) == len(timestamp)
    
    return pd.DataFrame({'DateTime': timestamp, 'Mean NO2': no2_values})

In [171]:
len(aoi)

43

In [ ]:
import time 

start = time.time()
aoi_no2 = list()

# Check length of provided aoi list OR dataframe
len_aoi = len(aoi)

# Define how many AOIs you want to process. For the demo we will use only one 
counter = 42

for idx in range(counter):
    aoi_dict = dict()
    
    aoi_dict['Country_BBox'] = aoi[0][idx]
    print("Processing: ", aoi_dict['Country_BBox'])
    x1 = int(aoi[1][idx])  # degree 
    y1 = int(aoi[2][idx])  # degree
    x2 = int(aoi[3][idx])  # degree
    y2 = int(aoi[4][idx])  # degree

    bbox = x1, y1, x2, y2
    
    timerange = ['2020-01-01', '2020-12-01']
    aoi_dict['NO2_2020'] = caculateNO2(bbox, timerange)
    shape = aoi_dict['NO2_2020'].shape
    nullCount = sum(aoi_dict['NO2_2020'].isna().sum())
    print(f"Shape of 2020 DF: {shape}, Count of Null values: {nullCount}".format(shape , nullCount ))
    
    timerange = ['2019-01-01', '2019-12-01']
    aoi_dict['NO2_2019'] = caculateNO2(bbox, timerange)
    shape = aoi_dict['NO2_2019'].shape
    nullCount = sum(aoi_dict['NO2_2019'].isna().sum())
    print(f"Shape of 2019 DF: {shape}, Count of Null values: {nullCount}".format(shape , nullCount ))    
    
    timerange = ['2018-03-01', '2018-12-01']
    aoi_dict['NO2_2018'] = caculateNO2(bbox, timerange)
    shape = aoi_dict['NO2_2018'].shape
    nullCount = sum(aoi_dict['NO2_2018'].isna().sum())
    print(f"Shape of 2018 DF: {shape}, Count of Null values: {nullCount}".format(shape , nullCount ))

    aoi_no2.append(aoi_dict)

end = time.time()
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)
#print(len(aoi_no2))

In [168]:
import pickle 
print(len(aoi_no2))
with open('aoi_list_no2.pkl', 'wb') as f:
    pickle.dump(aoi_no2, f)

In [169]:
with open('aoi_list_no2.pkl', 'rb') as fp:
    loaded = pickle.load(fp)

In [170]:
loaded

[{'Country_BBox': 'TUR',
  'NO2_2020':               DateTime  Mean NO2
  0  2020-01-04 12:00:00  0.000015
  1  2020-01-11 12:00:00  0.000023
  2  2020-01-18 12:00:00  0.000032
  3  2020-01-25 12:00:00  0.000025
  4  2020-02-01 12:00:00  0.000028
  5  2020-02-08 12:00:00  0.000032
  6  2020-02-15 12:00:00  0.000019
  7  2020-02-22 12:00:00  0.000027
  8  2020-02-29 12:00:00  0.000020
  9  2020-03-07 12:00:00  0.000022
  10 2020-03-14 12:00:00  0.000013
  11 2020-03-21 12:00:00  0.000018
  12 2020-03-28 12:00:00  0.000012
  13 2020-04-04 12:00:00  0.000013
  14 2020-04-11 12:00:00  0.000028
  15 2020-04-18 12:00:00       NaN
  16 2020-04-25 12:00:00  0.000013
  17 2020-05-02 12:00:00  0.000019
  18 2020-05-09 12:00:00  0.000019
  19 2020-05-16 12:00:00  0.000015
  20 2020-05-23 12:00:00  0.000009
  21 2020-05-30 12:00:00  0.000009
  22 2020-06-06 12:00:00  0.000016
  23 2020-06-13 12:00:00  0.000013
  24 2020-06-20 12:00:00  0.000021
  25 2020-06-27 12:00:00  0.000017
  26 2020-07-04 12